### Multi-node distributed training of *Hugging Face* models with *PyTorch*, *TorchDistributor*, and *Petastorm*

##### Here we perform fine tuning of a pre-trained NLP model from [*Hugging Face*](https://huggingface.co/). We use [*PyTorch*](https://pytorch.org/) as the Deep Learning framework for model training, with its [*Distributed Data Parallel*](https://pytorch.org/tutorials/beginner/dist_overview.html) approach and [*TorchDistributor*](https://www.databricks.com/blog/2023/04/20/pytorch-databricks-introducing-spark-pytorch-distributor.html) to provide PyTorch's distributed training infrastructure for *Spark* on Azure Databricks. [*Petastorm*](https://petastorm.readthedocs.io/en/latest/index.html) is used to integrate the distributed data layer on *Spark* with *PyTorch*.

Import the necessary packages.

In [0]:
import os
import shutil
import time
import random
import pickle
import numpy as np
import glob

import torch
from torch.optim import AdamW
import functools
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pyspark.ml.torch.distributor import TorchDistributor

from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec

import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler

from torch.nn.parallel import DistributedDataParallel as ddp

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  original_result = python_builtin_import(name, globals, locals, fromlist, level)
/databricks/python/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


Define the number of workers in the cluster and the location on the [*Microsoft Fabric Lakehouse*](https://learn.microsoft.com/en-us/fabric/data-engineering/lakehouse-overview) where the training and validation data were saved.

Notice that you should replace *YOUR_FABRIC_WORKSPACE_NAME* and *YOUR_FABRIC_LAKEHOUSE_NAME* with your own corresponding resource names.

In [0]:
num_workers = 8

lakehouse_data_folder = 'abfss://YOUR_FABRIC_WORKSPACE_NAME@msit-onelake.dfs.fabric.microsoft.com/YOUR_FABRIC_LAKEHOUSE_NAME.Lakehouse/Files'

Read the training data into a Spark DataFrame.

Notice that for reading from the Lakehouse without specifying any credentials is only possible in an Azure Databricks Premium Workspace. Please refer to [this document](https://learn.microsoft.com/en-us/fabric/onelake/onelake-azure-databricks) for more details.

In [0]:
sdf_tokens_train = spark.read.parquet(os.path.join(lakehouse_data_folder, 'prepared_data/tokens_train_data'))

display(sdf_tokens_train.select(sdf_tokens_train.columns).limit(10))

label,input_ids,attention_mask
1,"List(0, 28878, 34717, 738, 2156, 24234, 7223, 1008, 8, 8852, 30, 10, 278, 9, 21844, 4476, 2156, 24, 18, 10, 33589, 3569, 32592, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,"List(0, 102, 5128, 17888, 480, 10, 822, 7, 28, 14065, 3995, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
0,"List(0, 16472, 8648, 34, 4146, 9, 5, 7133, 22094, 9, 5, 986, 3493, 2156, 8, 24, 1326, 25, 600, 1236, 857, 4533, 1488, 3660, 5, 822, 31, 5, 124, 9, 10, 629, 636, 873, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,"List(0, 1970, 417, 42177, 16, 30, 117, 839, 10, 1969, 822, 2156, 53, 63, 10902, 10, 1307, 16426, 3724, 8, 5278, 3400, 9, 1461, 1571, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
0,"List(0, 118, 64, 11526, 42, 1569, 11, 130, 1617, 4832, 26081, 25805, 11405, 108, 159, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Read the validation data into a Spark DataFrame.

In [0]:
sdf_tokens_val = spark.read.parquet(os.path.join(lakehouse_data_folder, 'prepared_data/tokens_val_data'))

display(sdf_tokens_val.select(sdf_tokens_val.columns).limit(10))

label,input_ids,attention_mask
0,"List(0, 38908, 9, 667, 7, 11044, 103, 33072, 293, 2156, 646, 417, 14985, 742, 197, 28, 24097, 596, 10, 2173, 19, 39, 2959, 1249, 62, 11, 10, 1569, 42, 1099, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
0,"List(0, 102, 11676, 2125, 9, 173, 2156, 19, 18805, 8, 6054, 23, 5, 672, 9, 1159, 108, 2384, 8, 6197, 25816, 25, 14628, 3876, 25, 6066, 6690, 2156, 5345, 8, 3986, 1900, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
0,"List(0, 5481, 5313, 630, 75, 283, 203, 795, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
0,"List(0, 15204, 3343, 13837, 1669, 1974, 101, 10, 1294, 822, 30, 80, 1669, 54, 12420, 236, 7, 28, 2677, 40838, 326, 19112, 1696, 77, 51, 1733, 62, 479, 53, 51, 1762, 49, 23390, 18, 1007, 8, 5440, 13, 4617, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,"List(0, 1594, 5, 1468, 16, 7019, 8, 30889, 39802, 2156, 1236, 21625, 1242, 36740, 7, 3866, 102, 18, 40524, 13659, 24823, 368, 149, 39, 304, 9, 5, 2280, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)","List(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Here we set a folder in the Databricks File System to be used by *Petastorm* as a cache for intermediate data. We then create the [*Petastorm SparkDatasetConverter*](https://petastorm.readthedocs.io/en/latest/api.html#module-petastorm.spark.spark_dataset_converter) objects that we later use for reading data from *Spark DataFrame* obects into *PyTorch DataLoader* objects.

In [0]:
# Set a cache directory on DBFS FUSE for intermediate data.
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/petastorm/cache')

converter_train = make_spark_converter(sdf_tokens_train, dtype=None)
converter_val = make_spark_converter(sdf_tokens_val, dtype=None)

/databricks/python/lib/python3.10/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
The median size 98879 B (< 50 MB) of the parquet files is too small. Total size: 793249 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20230608170358-appid-app-20230608151251-0000-8c3cf8a1-1913-40fb-98a2-8a82af50c1ae/part-00000-tid-1141359304960746118-30600043-830c-4875-9d80-dd9b6643749d-116-1-c000.parquet, ...
The median size 15754 B (< 50 MB) of the parquet files is too small. Total size: 125078 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20230608170401-appid-app-20230608151251-0000-77ca230e-2ae8-492d-920d-8e34ed49ff9e/part-00003

Here is where we define the main function of this notebook, which specifies the model fine tuning procedure that uses the [*Fully Sharded Data Parallel*](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html) approach in PyTorch.

In [0]:
def finetune_model():

    # local_rank  refers to the rank of the current process within a single node (machine)
    # rank refers to the current process in the context of the entire cluster (all participating nodes)
    # world_size is the size of the cluster in number of nodes
    # these values will be set by TorchDistributor later when submitting the job
    ######
    local_rank = int(os.environ['LOCAL_RANK'])
    rank = int(os.environ['RANK'])
    world_size = int(os.environ['WORLD_SIZE'])
    ######

    dist.init_process_group('nccl', rank=rank, world_size=world_size)
    
    # rank 0 is the node that coordinates inter-node communication in a DDP cluster
    # when we need to run something only in a single node in the cluster, we usually use the rank 0 node
    ######
    model_folder = '/dbfs/model_outputs_ddp'
    if rank == 0:
        if os.path.isdir(model_folder):
            shutil.rmtree(model_folder)
        os.makedirs(model_folder)
        print(f'Cluster size: {world_size} GPUs')
    ######
    
    # set the GPU device for PyTorch
    ######
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        torch.cuda.set_device(local_rank)
        
    torch.cuda.empty_cache()
    ######
    
    # set random seeds for reproducibility
    ######
    seed_val = 111
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    ######

    # here we instantiate the pre-trained model from Hugging Face
    # then we instantiate the optimizer and scheduler for model training
    ######
    model_type = 'roberta-base'
    max_length = 128
    hidden_dropout_prob = 0.2
    attention_probs_dropout_prob = 0.3
    num_labels = 2
    batch_size = 48
    learning_rate = 2e-6 * world_size
    num_epochs = 100

    config = AutoConfig.from_pretrained(model_type, hidden_dropout_prob=hidden_dropout_prob,
                                        attention_probs_dropout_prob=attention_probs_dropout_prob, num_labels=num_labels)
    model = AutoModelForSequenceClassification.from_pretrained(model_type, config=config)
    model = model.to(device)

    adam_epsilon = 1e-8
    no_decay = ['bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
       'weight_decay_rate': 0.2},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
       'weight_decay_rate': 0.0}
    ]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    
    total_steps = len(converter_train) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    ######
    
    # a simple function to get an elapsed time in minutes and seconds
    ######
    def epoch_time(start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs
    ######

    # initiate the variables to be used in the training loop
    ######
    train_losses = []
    val_losses = []

    best_loss = None
    early_stop_min_delta = 0
    early_stop_counter = 0
    early_stop_patience = 5
    flag_tensor = torch.zeros(1).to(device)
    avg_exec_time = torch.Tensor([0]).to(device)
    ######
    
    # we create the PyTorch DataLoader objects from the Petastorm SparkDatasetConverter objects we defined earlier
    with converter_train.make_torch_dataloader(cur_shard=rank, shard_count=world_size, batch_size=batch_size) as train_dataloader, \
    converter_val.make_torch_dataloader(cur_shard=rank, shard_count=world_size, batch_size=batch_size) as val_dataloader:
        
        # define the iterators on top of the PyTorch DataLoader objects, to get batch data from them
        ######
        train_dataloader_iter = iter(train_dataloader)
        train_steps = len(converter_train) // (batch_size * world_size)
        
        val_dataloader_iter = iter(val_dataloader)
        val_steps = max(1, len(converter_val) // (batch_size * world_size))
        ######

        # this is the main training loop, where we fine tune the pre-trained model
        ######
        for n in range(num_epochs):
            train_loss = 0
            val_loss = 0
            start_time = time.time()

            # this block runs one epoch over the training data, running forward and backward passes and updating the model weights
            # we gather validation losses from all nodes to compute their average for tracking purposes
            # the actual loss used by the backward pass is the individual loss on each node
            # the distributed optimizer takes care of averaging and synchronizing the model weights across all nodes
            ######
            model.train()
            for step in range(train_steps):
                batch = next(train_dataloader_iter)
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                train_loss += loss.data / train_steps
                dist.all_reduce(train_loss, op=dist.ReduceOp.SUM)
                train_loss = train_loss / world_size
            ######

            # this block runs one epoch over the validation data
            ######
            with torch.no_grad():
                model.eval()
                for step in range(val_steps):
                    batch = next(val_dataloader_iter)
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    val_loss += loss.data / val_steps
                    dist.all_reduce(val_loss, op=dist.ReduceOp.SUM)
                    val_loss = val_loss / world_size
            ######

            # here we report the training and validation losses, and the elapsed time per epoch
            # we also implement a basic early stopping srategy
            ######
            if rank == 0:
                print('\nTrain loss after itaration %i: %f' % (n+1, train_loss))
                print('Validation loss after itaration %i: %f' % (n+1, val_loss))
                end_time = time.time()
                epoch_mins, epoch_secs = epoch_time(start_time, end_time)
                avg_exec_time += epoch_mins*60 + epoch_secs
                print('Time: %im %is' % (epoch_mins, epoch_secs))
                
                if best_loss == None:
                    best_loss = val_loss
                elif best_loss - val_loss > early_stop_min_delta:
                    best_loss = val_loss
                    early_stop_counter = 0
                elif best_loss - val_loss < early_stop_min_delta:
                    early_stop_counter += 1
                    print(f'Early stopping counter {early_stop_counter} of {early_stop_patience}')
                    if early_stop_counter >= early_stop_patience:
                        print('Early stopping')
                        flag_tensor = torch.ones(1).to(device)

            train_losses.append(train_loss.cpu())
            val_losses.append(val_loss.cpu())

            dist.all_reduce(flag_tensor, op=dist.ReduceOp.SUM)
            if flag_tensor == 1:
                break
    
    dist.all_reduce(avg_exec_time, op=dist.ReduceOp.SUM)
    ######

    # at the end of training, we save the trained model files and tracked losses
    ######
    if rank == 0:
        print('Saving best model to %s' % model_folder)

        cpu_state = model.state_dict()
        save_name = model_folder + '/roberta-base-finetuned' + '_' + str(n)
        torch.save(cpu_state, save_name)
        with open(model_folder + '/train_losses.pkl', 'wb') as f:
            pickle.dump(train_losses, f)
        with open(model_folder + '/val_losses.pkl', 'wb') as f:
            pickle.dump(val_losses, f)

        avg_exec_time = avg_exec_time / n
        print('Average execution time per epoch: %d seconds' % int(avg_exec_time))
    ######

    dist.barrier()
    dist.destroy_process_group()

    return avg_exec_time

We use the [*TorchDistributor*](https://www.databricks.com/blog/2023/04/20/pytorch-databricks-introducing-spark-pytorch-distributor.html) class to submit the Spark job that runs the model training function.

In [0]:
run_output = TorchDistributor(num_processes=8, local_mode=False, use_gpu=True).run(finetune_model)

Started distributed training with 8 executor proceses
/databricks/python/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem
/databricks/python/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem
/databricks/python/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem
/databricks/python/lib/python3.10/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.


In [0]:
print('Average execution time per epoch: %d seconds' % int(run_output.cpu()))

Average execution time per epoch: 22 seconds


Delete the *Petastorm* cache files defined earlier.

In [0]:
converter_train.delete()
converter_val.delete()

Test loading the saved model.

In [0]:
model_type = 'roberta-base'
max_length = 128
hidden_dropout_prob = 0.
attention_probs_dropout_prob = 0.
num_labels = 2

config = AutoConfig.from_pretrained(model_type, hidden_dropout_prob=hidden_dropout_prob,
                                    attention_probs_dropout_prob=attention_probs_dropout_prob, num_labels=num_labels)
model = AutoModelForSequenceClassification.from_pretrained(model_type, config=config)

model_folder = '/dbfs/model_outputs_ddp'
PATH = glob.glob(model_folder + '/roberta-base-finetuned_*')[0]
model.load_state_dict(torch.load(PATH))
model.eval()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Here we mount our external Lakehouse location, which we use to copy the saved files to.

In [0]:
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

lakehouse_mount_point = '/mnt/lakehouse'
dbutils.fs.mount(source=lakehouse_data_folder, mount_point=lakehouse_mount_point, extra_configs=configs)

True

Copy the saved files from the Databricks File System to the Lakehouse.

In [0]:
model_dbfs_folder = 'model_outputs_ddp'
dbutils.fs.cp(model_dbfs_folder, os.path.join(lakehouse_mount_point, model_dbfs_folder), recurse=True)

True

Unmount the Lakehouse.

In [0]:
dbutils.fs.unmount(lakehouse_mount_point)

/mnt/lakehouse has been unmounted.


True